### In this notebook, we will upload huggingface dataset to OLTP (Postgres in this context) and also use it as OLAP Bronze Zone (just for simple)

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set up

In [1]:
import os
import sys

import datasets
import pandas as pd
import plotly.express as px
from datasets import load_dataset
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sqlalchemy import create_engine

sys.path.insert(0, "..")
sys.path.insert(1, "../../")

import src.visualization.setup
from src.utils.data_prep import parse_dt, handle_dtypes, insert_chunk_to_oltp
from src.config.dataset_config import dataset_config

load_dotenv("../.env")
datasets.logging.set_verbosity_error()

2024-12-16 23:17:21.340 | INFO     | src.config.dataset_config:<module>:19 - Dataset Path: dinhlnd1610/HM-Personalized-Fashion-Recommendations


### Controller

In [2]:
class Args(BaseModel):
    run_name: str = "002-simulate-oltp"
    testing: bool = True
    notebook_persist_dp: str = None
    hf_dataset_path: str = dataset_config.HF_DATASET_PATH
    hf_dataset_token: str = dataset_config.HF_TOKEN

    user_col: str = "customer_id"
    item_col: str = "article_id"
    interaction_col: str = "price"
    timestamp_col: str = "t_dat"

    # Number of days left out not being pushed into the OLTP so that later we can simulate having them as new data
    num_days_holdout: int = 30
    holdout_fp: str = "../data/holdout.parquet"

    # Output PostgreSQL table
    transaction_table_name: str = "transactions"
    customer_metadata_name: str = "customer_metadata"
    article_metadata_name: str = "article_metadata"
    
    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        if not self.testing:
            os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self


args = Args().init()

print(args.model_dump_json(indent=2, exclude={"hf_dataset_token"}))

{
  "run_name": "002-simulate-oltp",
  "testing": true,
  "notebook_persist_dp": "/home/dinhln/Desktop/MLOPS/recsys/HM-ScalableRecs/notebooks/data/002-simulate-oltp",
  "hf_dataset_path": "dinhlnd1610/HM-Personalized-Fashion-Recommendations",
  "user_col": "customer_id",
  "item_col": "article_id",
  "interaction_col": "price",
  "timestamp_col": "t_dat",
  "num_days_holdout": 30,
  "holdout_fp": "../data/holdout.parquet",
  "transaction_table_name": "transactions",
  "customer_metadata_name": "customer_metadata",
  "article_metadata_name": "article_metadata"
}


## Load transactions and metadata

In [4]:
transactions_raw = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="transactions",
    split="train",
    trust_remote_code=True,
    num_proc = 8
)

transactions_raw_df = transactions_raw.to_pandas()
transactions_raw_df.head(5)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [5]:
customer_metadata_raw = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="customers",
    split="train",
    trust_remote_code=True,
    num_proc = 8
)

customer_metadata_raw = customer_metadata_raw.to_pandas()
customer_metadata_raw.head(5)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [6]:
article_metadata_raw = load_dataset(
    args.hf_dataset_path,
    token= args.hf_dataset_token,
    name="articles",
    split="train",
    trust_remote_code=True,
    num_proc=8
)

article_metadata_raw = article_metadata_raw.to_pandas()
article_metadata_raw.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [7]:
transactions_df = transactions_raw_df.pipe(parse_dt).pipe(handle_dtypes)
transactions_df.dtypes

t_dat               datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object

In [8]:
px.bar(
    data_frame=transactions_df.groupby(pd.Grouper(key="t_dat", freq="D"))
    .size()
    .reset_index(name="count"),
    x="t_dat",
    y="count",
    title="Number of data points over time",
)

In [9]:
holdout_date = (
    transactions_df["t_dat"].max() - pd.to_timedelta(args.num_days_holdout, unit="d")
).strftime("%Y-%m-%d")
logger.info(f"{holdout_date=}")
to_insert_transation_df = transactions_df.loc[lambda df: df["t_dat"].lt(holdout_date)]
holdout_transaction_df = transactions_df.loc[lambda df: df["t_dat"].ge(holdout_date)]

2024-12-16 13:55:54.463 | INFO     | __main__:<module>:4 - holdout_date='2020-08-23'


## Insert transactions and metadata into OLTP

In [10]:
# PostgreSQL connection details
username = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")
schema = os.getenv("POSTGRES_OLTP_SCHEMA")

# Create a connection string and engine outside the function
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [11]:
from psycopg2.extensions import register_adapter, AsIs
import numpy as np
register_adapter(np.int64, AsIs)
register_adapter(np.float64, AsIs)
print(to_insert_transation_df.memory_usage(deep=True).sum())

4931814951


In [ ]:
#Insert transactions
insert_chunk_to_oltp(to_insert_transation_df, engine, schema= "oltp", table_name= args.transaction_table_name)

Ingesting chunks:   0%|          | 0/30633 [00:00<?, ?it/s]

In [13]:
#Insert customer metadata
insert_chunk_to_oltp(customer_metadata_raw, engine, schema= "oltp", table_name= args.customer_metadata_name)

Ingesting chunks:   0%|          | 0/1372 [00:00<?, ?it/s]

In [14]:
# Insert article metadata
insert_chunk_to_oltp(article_metadata_raw, engine, schema= "oltp", table_name= args.article_metadata_name)

Ingesting chunks:   0%|          | 0/106 [00:00<?, ?it/s]